# 필요한 패키지, 모듈 및 함수 불러오기

In [1]:
import requests, re, time
from bs4 import BeautifulSoup
import pandas as pd
import imageio
import imgaug as ia
import sqlite3

# 크롤링 함수

In [2]:
def get_cookies():
    return requests.get('http://www.angel.or.kr/index.php').cookies.get_dict()

def get_list(**kwargs):
    cookies = kwargs['cookies']
    page = kwargs['page']
    ski = kwargs['ski'] # 품종 번호

    headers = {
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Referer': f'http://www.angel.or.kr/index.php',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,th;q=0.6',
    }

    params = (
        ('code', 'dog'),
        ('page', page),
        ('ski', ski),
        ('sci', ''),
        ('sco', ''),
        ('sgu', ''),
        ('q', ''),
    )

    response = requests.get('http://www.angel.or.kr/index.php', headers=headers, params=params, cookies=cookies, verify=False)
    return response

def get_doginfo(**kwargs):
    cookies = kwargs['cookies']
    # page = kwargs['page']
    # ski = kwargs['ski'] # 품종 번호
    number = kwargs['number']

    headers = {
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Referer': 'http://www.angel.or.kr/index.php',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,th;q=0.6',
    }

    params = (
        ('code', 'dog'),
        ('number', number),
        # ('page', page),
        # ('ski', ski),
        ('sci', ''),
        ('sco', ''),
        ('sgu', ''),
        ('q', ''),
        ('style', 'webzine'),
        ('listType', ''),
    )

    response = requests.get('http://www.angel.or.kr/view.php', headers=headers, params=params, cookies=cookies, verify=False)
    return response

# 코드 실행 과정

In [86]:
cookies = get_cookies()

In [87]:
page = 1
ski = 28 # 품종

response = get_list(page=page, ski=ski, cookies=cookies)

soup = BeautifulSoup(response.text, 'html.parser')

In [88]:
# 품종 출력
names = ['BEAGLE', 'BULLDOG','CHIHUAHUEÑO', 'GOLDEN RETRIEVER',
         'KOREA JINDO DOG', 'MALTESE', 'POMERANIAN', 'POODLE',
         'SHIH TZU', 'WELSH CORGI', ]

values = [28, 24, 59, 1, 57, 15, 66, 50, 41, 54]

breed_value = dict(zip(names, values))
value_breed = dict(zip(values, names))

for e in soup.find_all("option", value=re.compile(r'\d+')):
    print(e.text[:-1].strip())

골든 리트리버
그레이트 덴
그레이트 피레니즈
그레이하운드
기타견종
닥스훈트
달마시언
도베르만
도사
동경견
라사 압소
라이카
래브라도 리트리버
로트바일러
말티즈
미니어쳐 핀셔
믹스견
바셋 하운드
버니즈 마운틴 독
베들링턴 테리어
벨지언 세퍼드 독
보더 콜리
보르조이
보스턴 테리어
복서
불 테리어
불독
브리터니 스패니얼
비글
비숑 프리제
사모예드
삽살개
샤 페이
세인트 버나드 독
셔틀랜드 쉽독
슈나우저
스코티쉬 테리어
시 츄
시바
시베리안 허스키
아메리칸 코커 스패니얼
아메리칸 핏 불 테리어
아키타
아프간 하운드
알래스칸 맬러뮤트
오스트레일리언 실키 테리어
올드 잉글리시 쉽독
요크셔 테리어
웨스트 하이랜드 화이트 테리어
웰시 코기
이탈리언 코로소 독
잉글리시 세터
잉글리시 코커 스패니얼
잭 러셀 테리어
저먼 세퍼드 독
제퍼니즈 스피츠
제퍼니즈 친
진돗개
차우 차우
치와와
캐리 블루 테리어
캐벌리어 킹 찰스 스패니얼
코튼 드 툴리어
콜리
티베튼 스패니얼
파피용
퍼그
페키니즈
페터데일 테리어
포메라니언
포인터(포인팅 독)
폭스 테리어
푸들
풀리
풍산개
프랜치 불독
휘펫


In [89]:
# 썸네일 추출
thumbnails = soup.find_all('img', src=re.compile(r'^\./report|\./images'))
thumbnails = ['http://www.angel.or.kr' + e['src'][1:] for e in thumbnails]
thumbnails

['http://www.angel.or.kr/report/dog/300/1641387888_946035853.jpg',
 'http://www.angel.or.kr/images/noimg300.png',
 'http://www.angel.or.kr/report/dog/300/1628423803_311701271.jpg',
 'http://www.angel.or.kr/report/dog/300/1626782653_1473490203.jpg',
 'http://www.angel.or.kr/report/dog/300/1623402832_2027682480.jpg',
 'http://www.angel.or.kr/report/dog/300/1619777993_1698092004.jpg',
 'http://www.angel.or.kr/report/dog/300/1614991947_1270783763.jpg',
 'http://www.angel.or.kr/report/dog/300/1614251169_324685796.jpg',
 'http://www.angel.or.kr/report/dog/300/1600266688_575833667.jpg',
 'http://www.angel.or.kr/report/dog/300/1594680523_1266340706.jpg',
 'http://www.angel.or.kr/report/dog/300/1591087181_1534841008.jpg',
 'http://www.angel.or.kr/report/dog/300/1589584090_404758809.jpg',
 'http://www.angel.or.kr/report/dog/300/1588473799_2133669711.jpg',
 'http://www.angel.or.kr/report/dog/300/1587035044_228469501.jpg',
 'http://www.angel.or.kr/images/noimg300.png',
 'http://www.angel.or.kr/rep

In [90]:
# 각 정보들의 고유 번호
infos = soup.find_all('a', href=re.compile(r'^poster.php[.]*'))

p = re.compile(r'number=([\d]+)')

image = [p.search(e['href']).group(1) for e in infos]
num_list = [p.search(e['href']).group(1) for e in infos]
num_list

['47002',
 '46921',
 '46385',
 '46282',
 '46080',
 '45938',
 '45684',
 '45643',
 '44938',
 '44535',
 '44275',
 '44204',
 '44146',
 '44076',
 '44055',
 '43774',
 '43682',
 '43670',
 '43270',
 '43145']

In [91]:
# 정보 추출
soup_ = BeautifulSoup(get_doginfo(number=num_list[0], page=page, ski=ski, cookies=cookies).text)
soup_

info = soup_.tbody.find_all('tr')
info_type = ['동물', '날짜', '장소']

for t, e in zip(info_type, info[:3] + info[4:5]):
    print(f"{t}: {e.find_all('td')[-1].text.strip()}")

동물: 비글 / 성별모름 / 나이모름
날짜: 2022-01-05
장소: 서울특별시 용산구 용산 KCC 스위첸 아파트 앞


# 실제 크롤링 코드

In [71]:
# 품종 출력
names = ['BEAGLE', 'BULLDOG','CHIHUAHUEÑO', 'GOLDEN RETRIEVER',
         'KOREA JINDO DOG', 'MALTESE', 'POMERANIAN', 'POODLE',
         'SHIH TZU', 'WELSH CORGI', ]

values = [28, 24, 59, 1, 57, 15, 66, 50, 41, 54]

breed_value = dict(zip(names, values))
value_breed = dict(zip(values, names))

columns=['품종', '상태', '정보', '날짜', '장소', 'number', 'thumbnail']
sql = """CREATE TABLE IF NOT EXISTS test_table (
    품종 TEXT,
    상태 TEXT,
    정보 TEXT,
    날짜 TEXT,
    장소 TEXT,
    number TEXT,
    thumbnail TEXT
);"""
with sqlite3.connect('test.db') as conn:
    cur = conn.cursor()
    result = cur.execute(sql)

for name in names:
    cookies = get_cookies()
    ski = breed_value[name]
    page = 1

    # df = pd.DataFrame(data=[], columns=columns)

    while page:
        response = get_list(page=page, ski=ski, cookies=cookies)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 썸네일 추출
        thumbnails = soup.find_all('img', src=re.compile(r'^\./report|\./images'))
        thumbnails = ['http://www.angel.or.kr' + e['src'][1:] for e in thumbnails]
        thumbnails
        
        if thumbnails == []:
            page = 0
            continue
        
        # 각 정보들의 고유 번호
        infos = soup.find_all('a', href=re.compile(r'^poster.php[.]*'))
        p = re.compile(r'number=([\d]+)')

        image = [p.search(e['href']).group(1) for e in infos]
        num_list = [p.search(e['href']).group(1) for e in infos]
        num_list
        
        for i, number in enumerate(num_list):
            print(f'\r{name} - page({page}) - {i+1} / {len(thumbnails)} ({number})', end='')
            # 이미 저장된 정보는 넘어간다.
            sql = 'SELECT number FROM test_table where number=?;'
            with sqlite3.connect('test.db') as conn:
                cur = conn.cursor()
                result = cur.execute(sql, (number, ))

            if result.fetchall() == []:
                # 정보 추출
                soup_ = BeautifulSoup(get_doginfo(number=number, page=page, ski=ski, cookies=cookies).text)
                soup_

                state = soup_.tbody.td.text[:2]
                
                info_type = ['동물', '날짜', '장소']
                
                row_data = [value_breed[ski], state] + \
                [e.find_all('td')[-1].text.strip() for _, e in zip(info_type, soup_.tbody.find_all('tr')[:3])] + \
                    [number, thumbnails[i]]
                
                # 출력
                # image = imageio.imread(thumbnails[i])
                # ia.imshow(image)
                # for t, e in zip(info_type, soup_.tbody.find_all('tr')[:3]):
                #     print(f"{state}{t}: {e.find_all('td')[-1].text.strip()}")

                sql = f'INSERT INTO test_table VALUES ({",".join(["?"]*len(columns))});'
                with sqlite3.connect('test.db') as conn:
                    cur = conn.cursor()
                    result = cur.execute(sql, row_data)
                
                # df = df.append(dict(zip(columns, row_data)), ignore_index=True)
                time.sleep(1)
        
        page += 1

WELSH CORGI - page(5) - 15 / 15 (36107)

In [20]:
# with sqlite3.connect('test.db') as conn:
#     df.to_sql('test_table',
#               con=conn,
#               index=False,
#               if_exists='replace',
#               chunksize=1000,
#               )

In [2]:
with sqlite3.connect('test.db') as conn:
    df = pd.read_sql('SELECT * FROM test_table', con=conn, )

In [3]:
df

,품종,상태,정보,날짜,장소,number,thumbnail
0,BEAGLE,목격,비글 / 성별모름 / 나이모름,2022-01-05,서울특별시 용산구 용산 KCC 스위첸 아파트 앞,47002,http://www.angel.or.kr/report/dog/300/16413878...
1,BEAGLE,목격,비글 / 성별모름 / 나이모름,2021-12-3,충청북도 충주시 충주 건대병원4거리 ksgi 앞,46921,http://www.angel.or.kr/images/noimg300.png
2,BEAGLE,목격,비글 / 암컷 / 나이모름,2021-08-08,서울특별시 강북구 오패산로 207부근,46385,http://www.angel.or.kr/report/dog/300/16284238...
3,BEAGLE,실종,비글 / 수컷 / 2살 / 이름(아롱이),2021-7-11,"전라북도 익산시 인화동 남부 다락방, 빨강포차 근처",46282,http://www.angel.or.kr/report/dog/300/16267826...
4,BEAGLE,목격,비글 / 성별모름 / 나이모름,2021-6-11,경상북도 경주시 아화리 474-11 부근,46080,http://www.angel.or.kr/report/dog/300/16234028...
...,...,...,...,...,...,...,...
3613,WELSH CORGI,실종,웰시 코기 펨브로크 / 성별모름 / 1살미만 / 이름(고기야채),2017-03-12,경상북도 경주시 와산교 다리를 지나는 동네에서 없어졌습니다,37451,http://www.angel.or.kr/report/dog/300/14895050...
3614,WELSH CORGI,구조,웰시 코기 카디건 / 수컷 / 1살,2017-2-8,부산광역시 사하구 장림시장9길54 무지개빌라2동앞,37288,http://www.angel.or.kr/images/noimg300.png
3615,WELSH CORGI,실종,웰시 코기 펨브로크 / 수컷 / 3살 / 이름(예스),2016-12-29,경상북도 영주시 영주여객부근,37189,http://www.angel.or.kr/report/dog/300/14846363...
3616,WELSH CORGI,실종,웰시 코기 펨브로크 / 암컷 / 1살미만 / 이름(박스),2016-11-05,전라북도 전주시 완산구 아름다운웨딩홀 앞 나무갤러리,36862,http://www.angel.or.kr/report/dog/300/14793709...


In [ ]:
'INSERT INTO test_table (latitude,longitude) VALUES ({lat},{lng}) WHERE 장소={str(row[4])};'